![image-3.png](https://github.com/cafawo/Derivatives/blob/main/figures/fscampus_small.png?raw=1)

# Derivatives

**Prof. Dr. Fabian Woebbeking**</br>
Assistant Professor of Financial Economics

IWH - Leibniz Institute for Economic Research</br>
Frankfurt School of Finance & Management

fabian.woebbeking@dozent.frankfurt-school.de

## Case 3: Fixed Income

Case description ...

### Part 1: Basics

The following 3 zero bonds are given:

| Maturity | Price | Redemption |
|:--------:|:-----:|:----------:|
|    1Y    | 95.24 |     100    |
|    2Y    | 89.00 |     100    |
|    3Y    | 81.63 |     100    |

(Table generated with [TablesGenerator](https://www.tablesgenerator.com/markdown_tables))

#### Task: Calculate spot and forward rates.

In [156]:
# Trivially
discount_factors = [.9524, .8900, .8163]
print(discount_factors)

[0.9524, 0.89, 0.8163]


In [157]:
# Spot rates

def spot_rate(discount_factor, maturity):
    # Price in %
    return (1/discount_factor) ** (1 / maturity) - 1

spot_rates = [spot_rate(p, T) for p, T in [(.9524, 1),(.8900, 2), (.8163, 3)]]
spot_rates_formatted = [f"{r*100:.2f}%" for r in spot_rates]
print("\n".join(spot_rates_formatted))

5.00%
6.00%
7.00%


In [158]:
# Forward rates

# In 1 for 1
r_12 = (1 + spot_rates[1])**2 / (1 + spot_rates[0]) - 1
print(f"r_12 = {r_12*100:.2f}%")
# In 2 for 1
r_12 = (1 + spot_rates[2])**3 / (1 + spot_rates[1])**2 - 1
print(f"r_23 = {r_12*100:.2f}%")
# In 1 for 2
r_12 = ( (1 + spot_rates[2])**3 / (1 + spot_rates[0])**1 )**(1/2) - 1
print(f"r_13 = {r_12*100:.2f}%")

r_12 = 7.01%
r_23 = 9.03%
r_13 = 8.02%


#### Task: Price a three year 10% bond. Calculate the IRR and explain the relationship of the spot rates and the IRR.

In [159]:
def fair_price(coupon):
    # sum(discount_factors) is an annuity factor
    pv_cpn = coupon * sum(discount_factors)
    # the last discount factor [-1] discounts the face value
    pv = pv_cpn + discount_factors[-1]  
    return pv

print(f'Fair price 10% 3Y bond: {fair_price(0.1)*100:.4f}')

Fair price 10% 3Y bond: 108.2170


YTM is a weighted average of the spot rates.

#### Task: Evaluate with the same curve, a three year floater against the one year rate (bond base).

In [160]:
pv_float = 1 - discount_factors[-1]
pv_fix = discount_factors[-1]
print(f"PV floater = {(pv_float + pv_fix) * 100:.2f}%")

PV floater = 100.00%


#### Task: Solve analytically and calculate the par coupon rate for a three year bond.

Par coupon bond must fulfill

$$\begin{aligned}
100\% &= C \sum^T_{t=1} DF_t + DF_T\\
C &= \frac{100\% - DF_T}{\sum^T_{t=1} DF_t}  \\
\end{aligned}$$

In [161]:
# Par cpn bond rate
par_cpn_rate = (1 - discount_factors[-1]) / sum(discount_factors)
print(f"Par coupon: {par_cpn_rate*100:.2f}%")
# Check numerically
print(f'Fair price {par_cpn_rate*100:.2f}% 3Y bond: {fair_price(par_cpn_rate)*100:.2f}%')

Par coupon: 6.91%
Fair price 6.91% 3Y bond: 100.00%


### Part 2: Asset-/ liability-swap

#### Task: Calculate the level of the floating debt (liability swap).

Power has issued a five year bond with a coupon of 2% and a price of 100.50. The five year IRS is quoted 2,00 – 2,05 against six month EURIBOR. 

In [162]:
from scipy.optimize import minimize

def YTM(price, coupon, maturity):
    # calculate PV from irr
    def YTM_price(irr, coupon, maturity):
        T = maturity
        annuity_factor = (1 - (1 + irr) ** -T) / irr
        p = coupon * annuity_factor + 1 / (1 + irr) ** T
        return p
    # target function to be minimized
    def NPV2(irr):
        return (price - YTM_price(irr, coupon, maturity)) ** 2
    # set NPV = 0 and return both PVs and the irr
    opt = minimize(NPV2, x0=0.01, bounds = ((None, None),))
    return opt.x[0]

ytm_21 = YTM(100.5/100, 2/100, 5)
print(f'YTM = {ytm_21 * 100:.2f}%')
print(f'Liability swap level = Euribor {(ytm_21 - 0.02) * 10000:.0f} basis points')

YTM = 1.89%
Liability swap level = Euribor -11 basis points


#### Task: Calculate the asset swap.

The Investor has bought a five year AA Bond with a coupon of 3% for 101,00. The 5 Y IRS quotes 2,00 -2,05 against 6M Euribor. 

In [163]:
ytm_22 = YTM(101.0/100, 3/100, 5)
print(f'YTM = {ytm_22 * 100:.2f}%')
print(f'Asset swap level = Euribor {(ytm_22 - 0.0205) * 10000:.0f} basis points')

YTM = 2.78%
Asset swap level = Euribor 73 basis points


### Part 3: Single curve

The following swap rates are given: 

| Maturity | 12M Euribor | 
|:--------:|:-----------:|
|    1Y    |    4.00%    | 
|    2Y    |    5.00%    | 
|    3Y    |    6.00%    | 

(Tables generated with [TablesGenerator](https://www.tablesgenerator.com/markdown_tables))

#### Task: Bootstrap discount factors and spot rates using the single curve approach.

In [164]:
def swap_bootstrap(swap_rates):
    discount_factors = []
    for s in swap_rates:
        # See Slide 61 
        DF_T = (1 - s * sum(discount_factors)) / (1 + s)
        discount_factors.append(DF_T)
        
    zero_rates = []
    for T, DF_T in enumerate(discount_factors):
        # Python starts counting at 0, hence, +1
        T = T + 1
        # See Slide 48
        r_T = (1 / DF_T)**(1 / T) - 1
        zero_rates.append(r_T)
        
    return [discount_factors, zero_rates]

discount_factors, spot_rates = swap_bootstrap([0.04, 0.05, 0.06])
print(discount_factors)
print(spot_rates)

[0.9615384615384615, 0.9065934065934065, 0.8376529131246111]
[0.040000000000000036, 0.05025249489363426, 0.06082879324860291]


#### Task: Price a three year swap against the one year rate using the single curve approach.

In [165]:
def swap_pricing(discount_factors):
    # See Slide 66
    s_T = (1 - discount_factors[-1]) / sum(discount_factors)
    return s_T
    
print(f"3Y swap rate = {swap_pricing(discount_factors)*100:.2f}%")

3Y swap rate = 6.00%


#### Task: Price the forward swap in one for two. Show that the result is correct and compare it to the zero rate in one for two using the single curve approach. 

In [166]:
# Please note that discount_factors is a vector that starts with the first discount factor (t = 1)
s_13 = (discount_factors[0] - discount_factors[-1]) / sum(discount_factors[1::])

print(f"1X2 swap rate = {s_13*100:.2f}%")

1X2 swap rate = 7.10%


#### Task: Mark to market a swap, 8% against 12M Euribor with a remaining maturity of 3 years and EUR 100mln notional.

In [167]:
# Discounted swap rate differential
mtm = (0.08 - 0.06) * sum(discount_factors) * 100e+6
print(f"Receiver MTM = EUR {mtm:,.2f} mln")

# PV(Fix) - PV(Float)
mtm_alternative = 0.08 * sum(discount_factors) * 100e+6 - (1 - discount_factors[-1]) * 100e+6
print(f"Receiver MTM = EUR {mtm_alternative:,.2f} mln")


Receiver MTM = EUR 5,411,569.56 mln
Receiver MTM = EUR 5,411,569.56 mln


### Part 4: Multi curve

Consider a three year swap against the one year rate using the multi curve approach. The following swap rates are given: 

| Maturity | 12M Euribor | Eonia |
|:--------:|:-----------:|:-----:|
|    1Y    |    4.00%    | 3.30% |
|    2Y    |    5.00%    | 4.50% |
|    3Y    |    6.00%    | 5.30% |

(Tables generated with [TablesGenerator](https://www.tablesgenerator.com/markdown_tables))

#### Task: Calculate the spot rates.

In [168]:
eonia_discount_factors, eonia_spot_rates = swap_bootstrap([0.033, 0.045, 0.053])
print(eonia_discount_factors)
print(eonia_spot_rates)

[0.9680542110358181, 0.9152512540702279, 0.854876363104824]
[0.03299999999999992, 0.0452732436287433, 0.05365612701860023]


#### Task: Calculate the implied forward rates. 

(Hint: the one year spot rate for the swap cash flow is taken from the single curve approach.)


In [169]:
def multi_curve(swap_rates, eonia_rates):
    # bootstrap discount factors from eonia rates
    discount_factors, _ = swap_bootstrap(eonia_rates)
    # setup 
    SDF = 0 # sum of discount factors
    CD = 0  # discounted fixed leg until T
    LD = 0  # discounted float leg until T-1
    
    implied_forwards = []
    for C_T, DF_T in zip(swap_rates, discount_factors):
        SDF += DF_T  # add up discount factors
        CD = C_T * SDF  # C_T is an annuity
        L = (CD - LD) / DF_T  # calculate implied forward rate (see equation above)
        implied_forwards.append(L)  # round and save L
        LD += L * DF_T  # sum up discounted L (used in next iteration)
        
    return implied_forwards
    
implied_forward_rates = multi_curve([0.04, 0.05, 0.06], [0.033, 0.045, 0.053])
implied_forward_rates_formatted = [f"{r*100:.2f}%" for r in implied_forward_rates]
print(f"Implied forward rates: {implied_forward_rates_formatted}")

Implied forward rates: ['4.00%', '6.06%', '8.20%']


#### Task: Calculate the forward swap in one for two.


In [170]:
import numpy as np

implied_forwards = multi_curve([0.04, 0.05, 0.06], [0.033, 0.045, 0.053])
eonia_discount_factors, _ = swap_bootstrap([0.033, 0.045, 0.053])

# PV float leg, this was 1 - DF_T before:
PV_Var = sum(np.array(implied_forwards[1::]) * np.array(eonia_discount_factors[1::]))
# Swap rate
s_13m = PV_Var / sum(eonia_discount_factors[1::])

print(f"1X2 swap rate (multi curve) = {s_13m*100:.2f}%")

1X2 swap rate (multi curve) = 7.09%


### Part 5: Interest rate parity

#### Task: Calculate the one year (365 days) FX swap forward using covered interest rate parity. 
* Spot = 1.3193 
* USD interest = 1.05310%
* EUR interest = 1.4375%

In [171]:
spot = 1.3193  # EURUSD
r_usd = 0.010531
r_eur = 0.014375

fx_fwd = spot * (1 + r_usd * 365/360) / (1 + r_eur * 365/360)
print(f"EURUSD FX forward = {fx_fwd:.4f}")
print(f"EURUSD FX swap points = {fx_fwd - spot:.4f}")

EURUSD FX forward = 1.3142
EURUSD FX swap points = -0.0051


#### Task: Mark to market the following cross currency swap.

The term structure of interest rates is flat at 6% in USD and 8% in AUD (continuous rates). The current exchange rate is 0.64 USD per AUD. You pay 7% in AUD and receive 3% in USD, with principals being USD 15mln and AUD 25mln. Payments are exchanged annually with 3 payments left (3 years to maturity).

In [172]:
# Annuity factors
time = np.array([1,2,3])
annuity_factor_usd = sum(np.exp(- 0.06 * time))
annuity_factor_aud = sum(np.exp(- 0.08 * time))

# Payment legs
pv_usd = 0.03  * 15e+6 * annuity_factor_usd + 15e+6 * np.exp(-.06 * 3)
pv_aud = 0.07  * 25e+6 * annuity_factor_aud + 25e+6 * np.exp(-.08 * 3)

# PV difference
pv_ccy = pv_usd - pv_aud * 0.64

print(f"PV cross currency swap = USD {pv_ccy:,.2f}")


PV cross currency swap = USD -1,727,527.33
